# Modeling and Simulation in Python

Starter code for the orbit example

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

/usr/local/Cellar/python/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Earth falling into the sun

Here's a question from the web site [Ask an Astronomer](http://curious.astro.cornell.edu/about-us/39-our-solar-system/the-earth/other-catastrophes/57-how-long-would-it-take-the-earth-to-fall-into-the-sun-intermediate):

"If the Earth suddenly stopped orbiting the Sun, I know eventually it would be pulled in by the Sun's gravity and hit it. How long would it take the Earth to hit the Sun? I imagine it would go slowly at first and then pick up speed."

Here's a solution.

In [2]:
# Here are the units we'll need

s = UNITS.second
N = UNITS.newton
kg = UNITS.kilogram
m = UNITS.meter

<Unit('meter')>

In [3]:
# And an inition condition (with everything in SI units)

r_0 = 147e9

init = State(p = Vector(r_0, 0) * m,
             v = Vector(0, 0) * m / s )


,values
p,"[147000000000.0 meter, 0.0 meter]"
v,"[0.0 meter / second, 0.0 meter / second]"


In [4]:
# Making a system object

r_earth = 6.371e6 * m
r_sun = 695.508e6 * m

system = System(init=init,
                G=6.674e-11 * N / kg**2 * m**2,
                m1=1.989e30 * kg,
                r_final=r_sun + r_earth,
                m2=5.972e24 * kg,
                t_0=0 * s,
                t_end=1e7 * s)

,values
init,"p [147000000000.0 meter, 0.0 meter] ..."
G,6.674e-11 meter ** 2 * newton / kilogram ** 2
m1,1.989e+30 kilogram
r_final,701879000.0 meter
m2,5.972e+24 kilogram
t_0,0 second
t_end,10000000.0 second


In [20]:
# Here's a function that computes the force of gravity

def universal_gravitation(state, system):
    """Computes gravitational force.
    
    state: State object with distance r
    system: System object with m1, m2, and G
    """
    p, v = state
    unpack(system)
    
#    force = G * m1 * m2 / r**2
    force = (G * m1 * m2 / (p.mag)**2.0) * -1 * p.hat()
    return Vector(force)

In [23]:
universal_gravitation(init, system)

<Quantity([-3.6686486e+22 -0.0000000e+00], 'newton')>

In [19]:
Vector(universal_gravitation(init, system)).mag

<Quantity(3.6686485997501037e+22, 'newton')>

In [41]:
# The slope function

def slope_func(state, t, system):
    """Compute derivatives of the state.
    
    state: position, velocity
    t: time
    system: System object containing `g`
    
    returns: derivatives of p and v (equal to velocity and acceleration)
    """
    p, v = state
    unpack(system)    

    force = universal_gravitation(state, system)
    dpdt = v
    dvdt = -force / m2

    
    return dpdt.mag, dvdt.mag

In [51]:
# Always test the slope function!

slope_func(init, 0, system)

(<Quantity(0.0, 'meter / second')>,
 <Quantity(0.006143082049146188, 'newton / kilogram')>)

In [43]:
# Here's an event function that stops the simulation
# before the collision

def event_func(state, t, system):
    p, v = state
    return abs(p.x) - system.r_final

In [44]:
# Always test the event function!

event_func(init, 0, system)

<Quantity(146298121000.0, 'meter')>

In [45]:
# Finally we can run the simulation

results, details = run_ode_solver(system, slope_func, events=event_func)
details

ValueError: `y0` must be 1-dimensional.

In [46]:
# Here's how long it takes...

t_final = get_last_label(results) * s

NameError: name 'results' is not defined

In [47]:
# ... expressed in units we understand

t_final.to(UNITS.day)

NameError: name 't_final' is not defined

In [49]:
# Before plotting, we run the simulation again with `t_eval`

ts = linspace(t_0, t_final, 201)
results, details = run_ode_solver(system, slope_func, events=event_func, t_eval=ts)

NameError: name 't_final' is not defined

In [50]:
# Scaling the time steps to days

results.index /= 60 * 60 * 24

NameError: name 'results' is not defined

In [51]:
# Scaling the distance to million km

r = results.r / 1e9;

NameError: name 'results' is not defined

In [52]:
# And plotting

plot(r, label='r')

decorate(xlabel='Time (day)',
         ylabel='Distance from sun (million km)')

NameError: name 'r' is not defined

In [9]:
init = State(position = Vector(0, 149.6e9) * m, velocity = Vector(0,0)*m/s )

,values
position,"[0.0 meter, 149600000000.0 meter]"
velocity,"[0.0 meter / second, 0.0 meter / second]"


In [10]:
# Making a system object

r_earth = 6.371e6 * m
r_sun = 695.508e6 * m

system = System(init=init,
                G=6.674e-11 * N / kg**2 * m**2,
                m1=1.989e30 * kg,
                r_final=r_sun + r_earth,
                m2=5.972e24 * kg,
                t_0=0 * s,
                t_end=1e7 * s)

,values
init,"position [0.0 meter, 149600000000.0 ..."
G,6.674e-11 meter ** 2 * newton / kilogram ** 2
m1,1.989e+30 kilogram
r_final,701879000.0 meter
m2,5.972e+24 kilogram
t_0,0 second
t_end,10000000.0 second


<function modsim.Vector(*args, units=None)>

In [10]:
a = Vector(1,5)

<Quantity([1 5], 'dimensionless')>

In [11]:
a.mag

<Quantity(5.0990195135927845, 'dimensionless')>

In [35]:
run_ode_solver??

Signature: run_ode_solver(system, slope_func, **options)
Source:   
def run_ode_solver(system, slope_func, **options):
    """Computes a numerical solution to a differential equation.

    `system` must contain `init` with initial conditions,
    `t_0` with the start time, and `t_end` with the end time.

    It can contain any other parameters required by the slope function.

    `options` can be any legal options of `scipy.integrate.solve_ivp`

    system: System object
    slope_func: function that computes slopes

    returns: TimeFrame
    """
    # make sure `system` contains `init`
    if not hasattr(system, 'init'):
        msg = """It looks like `system` does not contain `init`
                 as a system variable.  `init` should be a State
                 object that specifies the initial condition:"""
        raise ValueError(msg)

    # make sure `system` contains `t_end`
    if not hasattr(system, 't_end'):
        msg = """It looks like `system` does not contain `t_end`
